In [131]:
import numpy as np
import cv2
import cvutils
from skimage import io
from numpy import linalg as LA
import scipy

In [132]:
# ! pip install opencv-python
# ! pip install cvutils
# ! pip install scikit-image


In [133]:
import os
from os import listdir
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
%matplotlib inline

In [134]:
# ! pip install opencv-python
# ! pip install cvutils
# !pip install scikit-image

In [135]:
def get_width_height_train_images():
    curr_path=os.getcwd()
    train_images_path=os.path.join(curr_path,"train_images")
    print(train_images_path)
    for folder in os.listdir(train_images_path):
        if ".DS_" in folder:
            continue
        train_image_folder=os.path.join(train_images_path,folder)
        for file in os.listdir(train_image_folder):
            if ".DS_" in file:
                continue
            train_image=os.path.join(train_image_folder,file)
#             im = Image.open(train_image)
            im = cv2.imread(train_image,0)
            width, height = im.shape
            print(width,height)
            return width,height
            
            

    

In [6]:

get_width_height_train_images()

/Users/amirulislam/projects/hbku/fall_2019/app_deep_learning/Face_Detection_MDC/train_images
2592 1944


(2592, 1944)

In [7]:
def resize_test_images():
    '''
    read the images and return a flattened matrix 
    containing the vectors
    '''
    
    # get te width and height of training image
    dim=get_width_height_train_images()
    print("target dim ",dim)
   
    curr_path=os.getcwd()
    
    test_images_path=os.path.join(curr_path,"test_images")
    # print(train_images_path)

    folders=listdir(test_images_path)

    # add all image matrix into this list as 1D
    global_file_count=0
    for folder in folders:
        if ".DS" in folder:
            continue

        folder_location=os.path.join(test_images_path,folder)
        files=listdir(folder_location)
        for file in files:
            file_name=os.path.join(folder_location,file)
#             print(file_name)
            
            if ".DS" in file:
                continue
            im = cv2.imread(file_name,0)            
            print("Old shape is ",im.shape) 
            dim=(dim[1],dim[0])
            im = cv2.resize(im, dim,interpolation = cv2.INTER_NEAREST)
            print("New shape is ",im.shape)
            cv2.imwrite(file_name,im)
            
#             image_pil = Image.open(file_name).convert('L')
#             image_pil = image_pil.resize((int(width), int(height)), Image.NEAREST)
#             image_pil.save(file_name)
            

            global_file_count+=1
    print("number of files read = ",global_file_count)

    #     print(image.shape)
    print("Done resizing")
    

In [8]:
resize_test_images()

/Users/amirulislam/projects/hbku/fall_2019/app_deep_learning/Face_Detection_MDC/train_images
2592 1944
target dim  (2592, 1944)
Old shape is  (2592, 1944)
New shape is  (2592, 1944)
number of files read =  1
Done resizing


In [9]:
def read_flatten(location,count):
    '''
    read the images and return a flattened matrix 
    containing the vectors
    '''
    train_images_path=location
    # print(train_images_path)
    width,height=get_width_height_train_images()

    folders=listdir(train_images_path)
    list_of_1d_matrix=[]

    # add all image matrix into this list as 1D
    global_file_count=0
    for folder in folders:
        if ".DS" in folder:
            continue

        folder_location=os.path.join(train_images_path,folder)
        files=listdir(folder_location)
        i=0
        for file in files:
            file_name=os.path.join(folder_location,file)
#             print(file_name)
            
            if ".DS" in file:
                continue
#             image_pil = Image.open(file_name).convert('L')
#             image = np.array(image_pil, 'uint8')
            im = cv2.imread(file_name,0)
            print(im.shape)
            plt.figure()
            plt.imshow(im)
            data = np.array(im)
            flattened = data.flatten()
            transposed=flattened.transpose()
            list_of_1d_matrix.append(transposed)
            
            plt.figure()
            plt.imshow(np.asarray(transposed.reshape(width,height)))

            
            i=i+1
            global_file_count+=1
            if i==count:
                break
    print("number of files read = ",global_file_count)

    #     print(image.shape)
    matrix_vectors=np.array(list_of_1d_matrix)
    plt.figure()
    plt.imshow(np.asarray(matrix_vectors[0].reshape(width,height)))

    return matrix_vectors
    

In [10]:
def calculate_mean_faces(image_matrix):
    '''
    calculate average vector
    subtract average from each vector
    '''
    matrix_vectors=image_matrix
    avg_vector=matrix_vectors.mean(0)
    matrix_vectors=(matrix_vectors-avg_vector)
    return matrix_vectors,avg_vector
    
    
    

In [11]:
def calculate_covariance_matrix(matrix):
    '''
    take any matrix and find its
    covariance matrix
    '''
#     return matrix.dot(matrix.transpose())
    return np.cov(matrix)

In [12]:
def get_eigen_values_sorted(matrix, percentage):
    eigen_values, eigen_vectors = LA.eig(matrix)
    
    
    
    idx = eigen_values.argsort()[::-1]   
    eigen_values = eigen_values[idx]
#     print("before sorting")
#     print(eigen_values)
#     print(eigen_vectors)
    
    print("Number of eigen values = ",len(idx))
    
    # to sort the eigen vectors
    list_of_eigen_vectors=[]
    for index in idx:
        list_of_eigen_vectors.append(eigen_vectors[index])
    eigen_vectors=np.array(list_of_eigen_vectors)

        

#     eigen_vectors = eigen_vectors[:,idx]
    
    ratio=percentage/100
    if eigen_vectors.shape[0]>=10:
        print("need to shorten")
        top=int(eigen_vectors.shape[0]*ratio)
        print("will take ",top)
        eigen_values=eigen_values[:top]
        eigen_vectors=eigen_vectors[:top]
    return eigen_values, eigen_vectors

    
    

In [13]:
def get_weight_vector_for_images(image_matrix,eigen_vectors):
# for each image
# we have to store a set of weights
    list_of_weights=[]
    i=0
    
    for mean_face in image_matrix:
    #     print(mean_face.shape)
        weights_for_this_face=[]
#         print("for face ",i)
        i=i+1
        for eigen_vector in eigen_vectors:
            print(eigen_vector.transpose(),"\n", mean_face)            
    #         print(eigen_vector.shape, mean_face.shape)
            weight=(eigen_vector.transpose().dot(mean_face))
        
            print("weights ",weight)
    #         print(mean_face)
            weights_for_this_face.append(weight)
        list_of_weights.append(weights_for_this_face)
    weight_vector=np.array(list_of_weights)
    return weight_vector


## Start here


## Step 1. Read image and flatten into 1d matrices

In [ ]:
curr_path=os.getcwd()
train_images_path=os.path.join(curr_path,"train_images")
count_of_files=3
matrix_vectors=read_flatten(train_images_path,count_of_files)
print(matrix_vectors.shape)


In [ ]:
width,height=get_width_height_train_images()
plt.figure()
plt.imshow(np.asarray(matrix_vectors[0].reshape(width,height)))


## Step 2. Calculate average of the matrix
## And calculate the mean faces

In [ ]:
mean_face_matrix,avg_vector=calculate_mean_faces(matrix_vectors)

In [ ]:
print(mean_face_matrix.shape)


In [ ]:
print(mean_face_matrix.max())
print(mean_face_matrix.min())

In [ ]:
plt.imshow(np.asarray(mean_face_matrix[0].reshape(width,height)))

## Step 3. Calculate covariance matrix

In [ ]:
covar_matrix=calculate_covariance_matrix(mean_face_matrix)

In [ ]:
print(covar_matrix)

In [ ]:
# print("{",end="")
# for row in covar_matrix:
#     print("{",end="")
#     for elem in row:
#         print(elem, end = ',')
#     print("}",end=",")
# print("}",end=",")

## Step 5. Calculate Eigen vectors and Eigen values

### consider the matrix [3,X3]

### compute the eigenvectors of  this small co variance matrix

In [ ]:

eigen_values,eigen_vectors=get_eigen_values_sorted(covar_matrix,100)

In [ ]:
print(eigen_values)



In [ ]:
print(eigen_vectors)

## Step 6. Calculate corresponsing Eigen vectors for AA(transpose)
#### now to calculate the eigen values and eigen vectors of the big matrix
#### Needs to be noted that eigen values are same
#### eigen vector is nothing but (7760X3)times(small_eigen_vector[3,])

In [ ]:
list_large_eigen_vector = []
for eigen_vector in eigen_vectors:
    print("Multiplying ",mean_face_matrix.transpose().shape," with ",eigen_vector.shape)
    new_eigen_vector=(mean_face_matrix.transpose()).dot(eigen_vector)
    print(new_eigen_vector)
    new_eigen_vector=new_eigen_vector / np.linalg.norm(new_eigen_vector)
    list_large_eigen_vector.append(new_eigen_vector)    
    

eigen_faces=np.array(list_large_eigen_vector)
print(eigen_faces.shape)    

In [ ]:
i=0

width,height=get_width_height_train_images()

for eigen_face in eigen_faces:
#     print(eigen_face.shape)
    plt.figure()
    plt.imshow(np.asarray(eigen_face.reshape(width,height)))
    i+=1
    if i ==10:
        break
print("showing first ",i," eigen faces out of ",eigen_faces.shape[0])

## Step 6. Mapping image into eigen faces

In [ ]:
weight_matrix=get_weight_vector_for_images(mean_face_matrix,eigen_faces)

In [ ]:
print(weight_matrix)

### Detour to check if I can get face back

In [ ]:
# Check if the weights give me back my face
# lets try for face 0

count=0
image_matrix=np.zeros(width*height,)

for face_weights in weight_matrix:
    if count==5:
        break
    i=0
#     print("Adding")
    for eigen_face in eigen_faces:
#         plt.figure()    
#         plt.imshow(np.asarray((face_weights[i]*eigen_face).reshape(243,320)))
        image_matrix+= eigen_face*face_weights[i]
        i+=1
    image_matrix+=avg_vector
    plt.figure()    
    plt.imshow(np.asarray(image_matrix.reshape(width,height)))
    count+=1
    
print("trying to get back first",count, "faces")
        
        
        

### End of Detour

### Test images

In [ ]:
## Step 0. Resize test images to be same as train images

In [ ]:
resize_test_images()

## Step 1. Given test image, search for closest match

In [ ]:
curr_path=os.getcwd()
test_images_path=os.path.join(curr_path,"test_images")
print(test_images_path)

test_matrix_vectors=read_flatten(test_images_path,1)




# files=listdir(test_images_path)
# # files.sort()



# list_of_1d_matrix=[]
# test_matrix_vectors=None
# # add all image matrix into this list as 1D

# for test_image in files:
# #     print(test_image)
#     file_name=os.path.join(test_images_path,test_image)    
    
# #     print(file_name)
#     if ".DS" in test_image:
#         continue
    
#     image_pil = Image.open(file_name).convert('L')
#     image = np.array(image_pil, 'uint8')
#     list_of_1d_matrix.append(image.flatten().transpose())
    
#     print(image.shape)
# test_matrix_vectors=np.array(list_of_1d_matrix)

print(test_matrix_vectors.shape)



## Step 2. Get Mean Face

In [ ]:
test_mean_faces=(test_matrix_vectors-avg_vector)
    

In [ ]:
print(test_mean_faces.shape)

## Step 3. Project into eigen space

In [ ]:
# need to create my own set of weights for eeach face
face_number=0
for test_mean_face in test_mean_faces:
    test_face_set_of_weights=[]
    print("for a face ",face_number)
    for eigen_face in eigen_faces:
        eigen_vector=eigen_face
#         print(eigen_vector.shape,test_mean_face.shape)
        test_face_set_of_weights.append(eigen_vector.transpose().dot(test_mean_face))
#     print(test_face_set_of_weights)
    
    #may or may not want to project, need to ask someone
    '''
    test_image_projection=np.zeros(77760,)
    i=0
    for weight in face_set_of_weights:
        print(weight,large_eigen_vectors[i].shape)
        test_image_projection+=weight*large_eigen_vectors[i]
        print("test image projection shape is ",test_image_projection.shape)
        i=i+1
    print("this is the test image ",test_image_projection)
    '''
    
    # now that we have got the weight set for this face,
    # let us compare it with the weights of the original faces
    i=0
    distances=[]
    for original_weight in weight_matrix:
#         print(i)
#         print("yo",test_face_set_of_weights)
        dist=np.linalg.norm(original_weight-test_face_set_of_weights)
        distances.append(dist)
        i=i+1

    print("distances are ",distances)
    print("minimum dist is ",min(distances))

    face_match=distances.index(min(distances))
    print("matches with face ",face_match)
    
    plt.figure()    
    plt.imshow(np.asarray(test_matrix_vectors[face_number].reshape(width,height)))

    plt.figure()    
    plt.imshow(np.asarray(matrix_vectors[face_match].reshape(width,height)))
    face_number+=1


        
        
    
        
    
    